In [87]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_original = pd.read_csv("./(V1.6)all_products_data_set.csv")
#df.insert(2, "color_info", "")
#df.insert(2, "pattern_info","")
#df.head()

In [2]:
import re
def tc(colors: str)->str:
    temp = (colors.lower().removeprefix("{").removesuffix("}"))
    res = re.sub('"', "", re.sub("/",",",temp))
    return res

In [3]:
import json
#f = open("./json_files/color_name_map.json")
#colors = json.load(f)
#len(colors.keys())

In [4]:
df = pd.read_csv("./look.csv")
for row in range(df.shape[0]):
    if not isinstance(df.loc[row]["colors"], str):
        df.loc[row,'colors'] = ""
        continue
    e = df.loc[row]["colors"]
    c = tc(e)
    df.loc[row,'colors'] = c

In [26]:

df.sample(10)
df.shape[0]

28426

In [6]:
for row in range(0,1):
    res = set()
    pattern = set()
    colors_str = df.loc[row]["colors"]
    if colors_str == "":
        continue
    colors_list = df.loc[row]["colors"].split(",")
    #print(row, colors_list)
    for item in colors_list:
        URL = "https://colornames.org/search/results/?type=exact&query="
        URL += item
        try:
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find_all("a", class_="button is-fullwidth freshButton")
            
            #the first result which has the highest vote
            content = results[0].find("span").text.strip()
            res.add(content)
        except:
            pattern.add(item)
    df.loc[row, "color_info"] = ",".join(res)
    df.loc[row, "pattern_info"] = ",".join(pattern)


In [34]:
df_original.columns

Index(['index', 'id', 'title', 'tags', 'images', 'gender', 'product_type',
       'product_type(modified)', 'main_category', 'sub_category',
       'match_most_similar_>80%_string', 'match_most_similar_>60%_string',
       'label_1st', 'label_2nd', 'label_3rd', 'buckets_num', 'buckets',
       'color_num', 'colors', 'url', 'raw_text'],
      dtype='object')

In [37]:
#df_original.columns.get_loc("color_num")

17

In [44]:
main_colors = ["Black", "Blue", "Brown", "Green", \
    "Grey", "Orange", "Pink", "Purple", \
    "Red", "Tan", "White", "Yellow",]
main_color = {}
count = 1
for c in main_colors:
    URL = "https://colornames.org/search/results/?type=exact&query="
    URL += c
    try:
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("a", class_="button is-fullwidth freshButton")
        
        #the first result which has the highest vote
        content = results[0].find("span").text.strip()
        main_color.update({count:content})
        count += 1
        
    except:
        print(c)
        

In [45]:
main_color

{1: 'Black (#000000)',
 2: 'Blue (#0000ff)',
 3: 'Brown (#964b00)',
 4: 'Green (#00ff00)',
 5: 'Grey (#808080)',
 6: 'Orange (#ff8000)',
 7: 'Pink (#ffc0cb)',
 8: 'Purple (#800080)',
 9: 'Red (#ff0000)',
 10: 'Tan (#d2b48c)',
 11: 'White (#ffffff)',
 12: 'Yellow (#ffff00)'}

In [49]:
df_original = df_original.rename(columns={"color_infor":"color_info"})

In [51]:
df_original.to_csv("(V1.6)all_products_data_set.csv", index=False)

In [60]:
s  = df_original.loc[87]["color_num"]
known = df_original.loc[87]["color_info"]
print(known, ";",s)
l = s.split(",")
print(l)
for n in l:
    print(int(n))

Femme Fatale (#a60019) ; 10, 11
['10', ' 11']
10
11


In [63]:
"Black".__contains__("Blackl")

False

In [72]:
type(df_original.loc[13558]["color_num"])

str

In [74]:
unknown = 0
for row in range(df_original.shape[0]):
    my = df_original.loc[row]["color_info"]
    ar = df_original.loc[row]["color_num"]
    if not isinstance(my, str) and ar == '0':
        unknown += 1
print(unknown, unknown/df_original.shape[0])

6797 0.23911208048969254


In [77]:
main_color

{1: 'Black (#000000)',
 2: 'Blue (#0000ff)',
 3: 'Brown (#964b00)',
 4: 'Green (#00ff00)',
 5: 'Grey (#808080)',
 6: 'Orange (#ff8000)',
 7: 'Pink (#ffc0cb)',
 8: 'Purple (#800080)',
 9: 'Red (#ff0000)',
 10: 'Tan (#d2b48c)',
 11: 'White (#ffffff)',
 12: 'Yellow (#ffff00)'}

In [95]:
#finish color_info from previous manual label
for row in range(df_original.shape[0]):
    color_num = df_original.loc[row]["color_num"]
    color_info = df_original.loc[row]["color_info"]
    #if we don't have the both information just jump to next one
    if not isinstance(color_info, str) and color_num == "0":
        continue
    if color_num == "0":
        continue
    
    l = color_num.split(",")
    
    if isinstance(color_info, str):
        res = (color_info).split(",")
        
    for n in l:
        n = int(n)
        try:
            color = main_color.get(n)
            if color == None:
                continue
            if not res.__contains__(color):
                res.append(color)
        except:
            print(n)
    
    df_original.loc[row, "color_info"] = ",".join(res)
            
        
    

In [97]:
df_original.to_csv("(V1.6)all_products_data_set.csv", index=False)